In [ ]:
import random
import numpy as np
import torch as T
import torch
# device = T.device("cuda") 
device = torch.device("cpu") 

Path = 'gen1_33'
EMBEDDING_SIZE = 150
SHAPE_EMBEDDING_SIZE = 6

In [ ]:
class FinalEmbedding:
    def __init__(self,x,y):
        self.x_data = x
        self.y_data = y
      
    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        preds = self.x_data[idx]
        trgts = self.y_data[idx] 
        sample = { 
        'predictors' : preds,
        'targets' : trgts
        }
        return sample

In [ ]:
api2indx = torch.load(Path + '/api2indx.pt')
api2indx

In [ ]:
def process_dataX(io_seq_tensors):
    io_seq = []
    for tensor_list in io_seq_tensors:        
        n0 = tensor_list[0]
        if(len(tensor_list) == 1):
            n1 = torch.zeros(n0.shape)
            n2 = torch.zeros(n0.shape)   
        elif(len(tensor_list) == 2):
            n1 = tensor_list[1]
            n2 = torch.zeros(n0.shape)
        elif(len(tensor_list) == 3):
            n1 = tensor_list[1]
            n2 = tensor_list[2]
        new_list = torch.stack((n0, n1, n2))
        io_seq.append(new_list)
    return(torch.stack(io_seq))

In [ ]:
indx2api = {}
EOS = '<eol>'

def process_dataY(api_seq_list):
    global indx2api
    global api2indx
    ''' Add <eol> to the dictionary '''
    indx2api = {v: k for k, v in api2indx.items()}
    if api2indx.get(EOS, -1) == -1:
        max_key = max(indx2api.keys())
        print(max_key)
        indx2api[max_key+1] = EOS
        api2indx[EOS] = max_key+1
    eos = api2indx[EOS]
    api_tensors = []
    for api_seq in api_seq_list:
        api0 = api_seq[0]
        if len(api_seq) == 1:
            api1 = eos
            api2 = eos
        elif len(api_seq) == 2:
            api1 = api_seq[1]
            api2 = eos
        elif len(api_seq) == 3:
            api1 = api_seq[1]
            api2 = api_seq[2]
        else:
            print('!!! Not supposed to be here')
        t = torch.tensor([api0, api1, api2])
        api_tensors.append(t)
    return(torch.stack(api_tensors))

In [ ]:
def shuffle_dataset(X_dataset, Y_dataset):

    print('len orig_dataset', len(X_dataset),  len(Y_dataset))
  
    length = len(X_dataset)

    idx = list(range(length))  # indices to all elements
    random.shuffle(idx)  # in-place shuffle the indices to facilitate random splitting
    data_idx = idx[:]

    x = []
    y = []

    x = [X_dataset[i] for i in data_idx]
    y = [Y_dataset[i] for i in data_idx]

    print(len(x), len(y))
    return(x,y)

In [ ]:
def dump_final_embedding():

    bat_size = 64

    for i in range(4,9):
        print(i)
        embed_file = Path + '/training_embeddings/' + str(i*10000) + '_training_embedding.pt'  
            
        print(embed_file)
        x, y = torch.load(embed_file)
        x, y = shuffle_dataset(x,y)
        pretrain_set_x = process_dataX(x)
        pretrain_set_y = process_dataY(y)
        print("Creating Final Datasets...")
        del x
        del y
    
        pretrain_ds = FinalEmbedding(pretrain_set_x, pretrain_set_y)
        pretrain_ldr = T.utils.data.DataLoader(pretrain_ds,batch_size=bat_size, shuffle=True)

        dest_file = Path + '/train_ldr_' + str(i) + '.pt'

        torch.save(pretrain_ldr, dest_file)
        print('saving in ', dest_file)
        del pretrain_set_x
        del pretrain_set_y
        del pretrain_ds
        del pretrain_ldr
        
dump_final_embedding()

In [ ]:
torch.save(api2indx, Path + '/api2indx.pt')

In [ ]:
api2indx

In [ ]:
import torch
from torch import nn

import numpy as np

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(RNNModel, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True, bidirectional=True)   
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim*2, output_size)
        
    
    def forward(self, x):
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)
        

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out1 = out.contiguous().view(-1, self.hidden_dim*2)
        out1 = self.fc(out1)
        
        return out1, hidden, out
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers*2, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden

In [ ]:
class FFNet(T.nn.Module):
  def __init__(self):
    super(FFNet, self).__init__()

    self.hid1 = T.nn.Linear(4*(EMBEDDING_SIZE+SHAPE_EMBEDDING_SIZE+1+2), 500)
    self.hid2 = T.nn.Linear(500, 250)
    self.hid3 = T.nn.Linear(250, 100)
    self.oupt = T.nn.Linear(100, len(api2indx))

    T.nn.init.xavier_uniform_(self.hid1.weight)
    T.nn.init.zeros_(self.hid1.bias)
    T.nn.init.xavier_uniform_(self.hid2.weight)
    T.nn.init.zeros_(self.hid2.bias)
    T.nn.init.xavier_uniform_(self.oupt.weight)
    T.nn.init.zeros_(self.oupt.bias)

    T.nn.Dropout(p=0.2)


  def forward(self, x):
    z1 = T.tanh(self.hid1(x))
    z2 = T.tanh(self.hid2(z1))
    z3 = T.tanh(self.hid3(z2))
    z = self.oupt(z3)  # no softmax: CrossEntropyLoss() 
    return (z, z3, z2, z1)


In [ ]:
# 0. get started
print("\nBegin predict  \n")
np.random.seed(1)
T.manual_seed(1)

In [ ]:
# Instantiate the model with hyperparameters
rnn_model = RNNModel(input_size=100, output_size=len(api2indx), hidden_dim=128, n_layers=2)
# We'll also set the model to the device that we defined earlier (default is CPU)
rnn_model = rnn_model.to(device)
# 2. create network
net = FFNet().to(device)

In [ ]:
def get_stored_dataset():

    ldrs = []

    for i in range(6,9):
        ldr_file = Path + '/train_ldr_' + str(i) + '.pt'
        print('ldr_file: ', ldr_file)
        ldrs.append(ldr_file)
    return(ldrs)

ldrs = get_stored_dataset()
print(ldrs)
    

In [ ]:
# Define hyperparameters
n_epochs = 20
lr=0.0001
bat_size = 64

# Define Loss, Optimizer
criterion2 = nn.CrossEntropyLoss()
optimizer2 = torch.optim.Adam(rnn_model.parameters(), lr=lr)

# 3. train model
max_epochs = n_epochs
ep_log_interval = 10
optimizer1 = T.optim.Adam(net.parameters(), lr=lr)

In [ ]:
import torch.nn as nn

print("\nbat_size = %3d " % bat_size)
print("loss = " + str(criterion2))
print("optimizer = Adam")
print("max_epochs = %3d " % max_epochs)
print("lrn_rate = %0.4f " % lr)
count = 0 
print("\nStarting train with saved checkpoints")
net.train()
rnn_model.train()

for epoch in range(0, max_epochs+1):
  epoch_loss = 0  # for one full epoch
  for index, ldr_path  in enumerate(ldrs):
    pretrain_ldr = torch.load(ldr_path)
    for (batch_idx, batch) in enumerate(pretrain_ldr):
      X = batch['predictors']  
      Y = batch['targets']   

      optimizer1.zero_grad()
      optimizer2.zero_grad()
      oupt, z3, z2, z1 = net(X.to(device))

      output, hidden, _ = rnn_model(z3) ##RNN
      
      output = output.to(device)
      target_seq = Y.to(device)
    
      loss = criterion2(output, target_seq.view(-1).long())
      loss.backward() # Does backpropagation and calculates gradients

      optimizer1.step()
      optimizer2.step() 
    
    del pretrain_ldr
  
  if epoch%2 == 0:
      print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
      print("Loss: {:.4f}".format(loss.item()))
      torch.save(net, Path + '/' + str(epoch) + '_train_net_model.pt')
      torch.save(rnn_model, Path + '/' + str(epoch) + '_train_rnn_model.pt')